# Mnist Dataset

![Image of Sample Mnist Dataset](https://camo.githubusercontent.com/2eefe56bdacb4d28015c435d0801abf00b95b95f/68747470733a2f2f636f72706f637261742e636f6d2f77702d636f6e74656e742f75706c6f6164732f323031342f31302f6669677572655f312e706e67)

## What is the Mnist Dataset?

The MNIST database of handwritten digits, available from this page, has a training set of 60,000 examples, and a test set of 10,000 examples. It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image.
It is a good database for people who want to try learning techniques and pattern recognition methods on real-world data while spending minimal efforts on preprocessing and formatting. (Source:http://yann.lecun.com/exdb/mnist/)

*The files from the dataset are also available from the website above but I have them saved here on github for easy access*

## Reading in the file

In [4]:
# Because the file is compressed (.gz) we have to unzip it we do this using a python library called gzip

import gzip

with gzip.open('Dataset/t10k-images-idx3-ubyte.gz','rb') as f:        
    file_content = f.read()
    
# If we do a simple type() function we can see what is file_content now

type(file_content)


bytes